In [1]:
!python -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1


In [2]:
# Install Pytorch & other libraries
%pip install -q tensorboard wandb 
 
# Install Hugging Face libraries
%pip install -q --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen-vl-utils"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pillow==9.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 184.5 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0


## 1. 데이터 전처리

과제물 사진과 과제물 정보를 주고 실제 정답을 맞추는 모델을 개발합니다.

이 모델은 선생님들이 학생들의 수행평가를 채점한다고 가정합니다. 본 프로젝트는 실제 교육 현장의 데이터와 차이가 있을 수 있으나, 자동 채점 시스템의 기본 원리와 가능성을 탐구하는 데 중점을 둡니다.

이번 예시에서는 Ko-SciecneQA 데이터셋을 사용할 건이중에서 이미지를 가지고 있는 6,218개의 데이터 중 시간 절약을 위해서 모두 사용하지는 않고 여기서 20%(1,243)만 사용하겠습니다.

이미지, 문제, 힌트을 기반으로 정답을 생성하도록 모델을 파인튜닝하려 합니다.
따라서 이미지, 문제, 힌트를 포함한 입력을 만들고, 이를 이용하여 정답을 찾아보겠습니다.

In [5]:
prompt= """질문: {korean_question}
선택지: {korean_choices}
힌트: {korean_hint}"""

system_message = "주어진 이미지와 질문을 바탕으로 답변하세요.\n이때 정답은 선택지 중 1개를 선택해야하며 힌트가 주어질 수 있습니다. 가장 적절한 답을 1개 선택하세요."

In [6]:
from datasets import load_dataset

# 데이터셋을 OpenAI 메시지 형식으로 변환하는 함수      
def format_data(sample):
   return {"messages": [
               {
                   # 시스템 프롬프트
                   "role": "system", 
                   "content": [{"type": "text", "text": system_message}], 
               },
               {
                   # 유저 프롬프트
                   "role": "user",  
                   "content": [
                       {
                           "type": "text",
                           "text": prompt.format(
                              korean_question=sample["korean_question"], 
                              korean_choices=sample["korean_choices"], 
                              korean_hint=sample["korean_hint"]
                              ),
                       },{
                           "type": "image", 
                           "image": sample["image"] 
                                if sample["image"] is not None else "", 
                       }
                   ],
               },
               {
                   # AI 어시스턴트 답변
                   
                   "role": "assistant", 
                   "content": [
                        {
                            "type": "text", 
                            "text": sample["answer_str"]
                        }
                    ], 
               },
           ],
       }

# 허브에서 데이터셋 로드 및 이미지가 존재하는 경우만 필터링
dataset = load_dataset("daje/Ko-SciecneQA", split="train")
dataset = dataset.filter(lambda example: example["image"] is not None)
dataset = [format_data(sample) for sample in dataset]

In [7]:
print('데이터의 개수:', len(dataset))

데이터의 개수: 6218


In [8]:
print('첫번째 데이터 출력:')
dataset[0]

첫번째 데이터 출력:


{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': '주어진 이미지와 질문을 바탕으로 답변하세요.\n이때 정답은 선택지 중 1개를 선택해야하며 힌트가 주어질 수 있습니다. 가장 적절한 답을 1개 선택하세요.'}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': "질문: 다음 주 중에서 가장 북쪽에 있는 곳은 어디인가요?\n선택지: ['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']\n힌트: "},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429>}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': '웨스트버지니아'}]}]}

In [9]:
print('힌트가 존재하는 데이터 출력:')
dataset[1500]

힌트가 존재하는 데이터 출력:


{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': '주어진 이미지와 질문을 바탕으로 답변하세요.\n이때 정답은 선택지 중 1개를 선택해야하며 힌트가 주어질 수 있습니다. 가장 적절한 답을 1개 선택하세요.'}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': "질문: 각 쌍의 자석 사이의 자기력을 생각해보세요. 다음 중 어떤 진술이 맞나요?\n선택지: ['두 쌍의 자기력 크기는 동일하다.', '쌍 1의 자기력 크기가 더 크다.', '쌍 2의 자기력 크기가 더 크다.']\n힌트: 아래 이미지는 두 쌍의 자석을 보여줍니다. 다른 쌍의 자석들은 서로 영향을 미치지 않습니다. 보여진 모든 자석은 같은 재료로 만들어졌지만, 크기와 모양이 다를 수 있습니다."},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=448x171>}]},
  {'role': 'assistant',
   'content': [{'type': 'text', 'text': '쌍 2의 자기력 크기가 더 크다.'}]}]}

In [10]:
train_dataset = dataset[:int(len(dataset) * 0.9)]
test_dataset = dataset[int(len(dataset) * 0.9):]

In [11]:
print('학습 데이터의 개수:', len(train_dataset))
print('테스트 데이터의 개수:', len(test_dataset))

학습 데이터의 개수: 5596
테스트 데이터의 개수: 622


## trl의 SFTTrainer를 이용한 파인 튜닝당하는 모듈입니다.

trl의 SFTTrainer를 사용해 모델을 파인튜닝합니다. SFTTrainer는 오픈소스 LLM과 VLM의 지도 파인튜닝을 매우 간단하게 만들어줍니다.  
SFTTrainer는 transformers 라이브러리의 Trainer를 상속받아서 로깅, 평가, 체크포인트 등 모든 기능을 지원하면서도 추가적인 편의 기능을 제공합니다.

이번 예시에서는 PEFT 기능을 사용할 예정입니다.  
PEFT 방법으로는 QLoRA를 사용할 건데, 이는 양자화와 LoRA 튜닝을 같이 사용하여 대규모 언어 모델의 메모리 사용량을 줄이는 기술입니다.

참고: 멀티모달 입력에 패딩이 필요하기 때문에 Flash Attention은 사용할 수 없습니다.  
Qwen 2 VL 72B 모델을 사용할 예정이지만, model_id 변수만 바꾸면 Meta AI의 Llama-3.2-11B-Vision, Mistral AI의 Pixtral-12B 등 다른 모델로도 쉽게 교체할 수 있습니다. bitsandbytes를 사용해 모델을 4비트로 양자화할 예정입니다.

참고: 모델이 클수록 더 많은 메모리가 필요합니다.  
VLM 학습을 위해 LLM, 토크나이저, 프로세서를 올바르게 준비하는 것이 매우 중요합니다. 프로세서는 특수 토큰과 이미지를 입력에 포함시키는 역할을 담당하는 모듈입니다.

In [12]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
 
# 허깅페이스에서 제공하는 Qwen 시리즈의 비전-언어 모델 ID
model_id = "Qwen/Qwen2-VL-7B-Instruct"
 
# 모델과 프로세서 로드
model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   # 가용한 GPU 메모리에 모델을 자동으로 최적 할당하는 설정
   device_map="auto",     
   # 메모리 효율과 연산 속도를 위해 bfloat16 형식의 부동소수점 정밀도 사용             
   torch_dtype=torch.bfloat16,                  
)
 
# 입력 텍스트와 이미지를 모델이 이해할 수 있는 형태로 변환하는 전처리기 로드
processor = AutoProcessor.from_pretrained(model_id)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

기존 LLM 템플릿
```python
<|im_start|>system
시스템 프롬프트<|im_end|>
<|im_start|>user
사용자의 질문<|im_end|>
<|im_start|>assistant
답변<|im_end|>
```

VLM 템플릿
```python
<|im_start|>system
시스템 프롬프트<|im_end|>
<|im_start|>user
사용자의 질문<|vision_start|>이미지<|vision_end|><|im_end|>
<|im_start|>assistant
답변<|im_end|>
```

## LoRA Config

In [13]:
from peft import LoraConfig

peft_config = LoraConfig(
       # 모델 가중치에 LoRA 업데이트를 적용하는 정도를 조절하는 스케일링 계수
       lora_alpha=128,
       # 과적합을 방지하기 위한 드롭아웃 비율 설정
       lora_dropout=0.05,
       # LoRA의 순위(rank) - 저차원 행렬의 차원을 결정
       r=256,
       # 편향(bias) 업데이트 여부 - 'none'은 편향을 업데이트하지 않음
       bias="none",
       # LoRA를 적용할 대상 모듈들 - 트랜스포머 모델의 주요 투영 레이어들
       target_modules=[
           "q_proj",    # Query 투영 레이어
           "up_proj",   # FFN 상향 투영 레이어
           "o_proj",    # Output 투영 레이어
           "k_proj",    # Key 투영 레이어
           "down_proj", # FFN 하향 투영 레이어
           "gate_proj", # FFN 게이트 투영 레이어
           "v_proj"     # Value 투영 레이어
       ],
       # 작업 유형 지정 - 인과적 언어 모델링(다음 토큰 예측)
       task_type="CAUSAL_LM",
)

In [14]:
from trl import SFTConfig
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

# SFTConfig를 통해 학습 설정을 정의
args = SFTConfig(
    # 학습된 모델과 체크포인트를 저장할 디렉터리 경로 및 리포지토리 ID
    output_dir="output_dir",
    # 전체 학습 에포크 수 (데이터셋을 몇 번 반복할지 설정)
    num_train_epochs=3,                                     
    # 각 장비(GPU)당 사용될 배치 사이즈 (메모리와 연관됨)
    per_device_train_batch_size=4,                          
    # 경사 누적 스텝 수 (이 횟수만큼 기울기를 누적한 후 업데이트)
    gradient_accumulation_steps=8,                          
    # 메모리 절약을 위한 gradient checkpointing 활성화 (메모리 최적화)
    gradient_checkpointing=True,                            
    # AdamW 옵티마이저 (fused 버전 사용으로 학습 속도 향상)
    optim="adamw_torch_fused",                              
    # 몇 스텝마다 로그를 출력할지 설정 (여기선 5 스텝마다 로그)
    logging_steps=5,                                        
    # 매 에포크마다 체크포인트 저장 설정
    save_strategy="epoch",                                  
    # 학습의 정도. 실험을 통해 결정하는 것을 권장
    learning_rate=1e-4,                                     
    # bfloat16 정밀도 사용 (메모리 절약 및 속도 향상)
    bf16=True,                                              
    # tf32 정밀도 사용 (NVIDIA GPU에서 학습 속도 향상)
    tf32=True,                                              
    # 기울기 클리핑을 위한 최대 기울기 값 (QLoRA 논문에서 추천된 값)
    max_grad_norm=0.3,                                      
    # 학습 초기에 학습률을 점진적으로 올리는 warmup 비율 (QLoRA 논문에서 추천된 값)
    warmup_ratio=0.03,                                      
    # 일정한 학습률 스케줄러 사용 (학습률이 변하지 않음)
    lr_scheduler_type="constant",                                            
    # TensorBoard를 통해 학습 상태를 모니터링
    report_to="tensorboard",                                
    # reentrant gradient checkpointing 설정 (비재진입 방식 사용)
    gradient_checkpointing_kwargs={"use_reentrant": False}, 
    # 데이터셋에서 텍스트 필드를 위한 더미 필드 (collator에서 필요)
    dataset_text_field="",                                  
    # collator에서 데이터셋 전처리를 건너뛰기 위한 설정
    dataset_kwargs={"skip_prepare_dataset": True},
    # 불필요한 열 삭제하지 않도록 설정 (학습 중 사용되지 않는 열이라도 유지)
    remove_unused_columns = False
)

In [15]:
# 텍스트와 이미지 쌍을 인코딩하기 위한 데이터 collator 함수 정의
def collate_fn(examples):
    # 각 예제에서 텍스트와 이미지를 추출하고, 텍스트는 채팅 템플릿을 적용
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

    # 텍스트를 토크나이징하고 이미지를 처리하여 일괄 처리(batch) 형태로 변환
    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)

    # labels로 사용할 input_ids 복사본 생성 후, 패딩 토큰을 -100으로 설정하여 손실 계산 시 무시하도록 함
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100  # 패딩 토큰 손실 계산 제외

    # 특정 이미지 토큰 인덱스는 손실 계산에서 무시 (모델에 따라 다름)
    if isinstance(processor, Qwen2VLProcessor):  
        # Qwen2VL 모델의 이미지 토큰 인덱스
        image_tokens = [151652, 151653, 151655]
    else:
        # 다른 모델에서 이미지 토큰 ID를 얻어 손실 계산에서 제외
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]
    
    # 손실 계산 시 이미지 토큰 인덱스를 무시하도록 설정
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100
    
    # 배치에 labels 추가 (손실 계산 시 사용)
    batch["labels"] = labels

    return batch

In [16]:
# 단일 예시 확인
example = dataset[0]  # 데이터셋의 첫 번째 아이템
print("단일 예시 데이터:")
print(example)

단일 예시 데이터:
{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '주어진 이미지와 질문을 바탕으로 답변하세요.\n이때 정답은 선택지 중 1개를 선택해야하며 힌트가 주어질 수 있습니다. 가장 적절한 답을 1개 선택하세요.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "질문: 다음 주 중에서 가장 북쪽에 있는 곳은 어디인가요?\n선택지: ['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']\n힌트: "}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429 at 0x7F90596326B0>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '웨스트버지니아'}]}]}


In [17]:
# collate_fn 테스트 (배치 크기 1로)
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("이미지 픽셀 형태:", batch["pixel_values"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 538])
어텐션 마스크 형태: torch.Size([1, 538])
이미지 픽셀 형태: torch.Size([1620, 1176])
레이블 형태: torch.Size([1, 538])


In [18]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0])

입력에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  54330,  31079,  85251,  90667,  21329,  80573,
        138520,  17877,  81718, 144059,  42039, 143604,  91145,    624,  12802,
        131866,  36055, 132760,  33704,  86038,  21329,  70943,    220,     16,
         59761,  18411,  86038, 129264, 130705,  10764,    252,    234,  28626,
         19969,  55673,  31079, 128732,  28733, 128472,     13, 130887, 135968,
        126550,  23573, 143603,  17877,    220,     16,  59761,  86038,  91145,
            13, 151645,    198, 151644,    872,    198, 128732,  51588,     25,
        126844,  55673,  70943,  56475, 130887, 139963, 132064,  19391,  64521,
         45130,    111,  33704, 139740,  31328,  19969,  35711,   5267,  14559,
         75132,  21329,     25,   2509, 144025,  53189,  79004,  21329,  83036,
         52959,    516,    364, 126746,  12802,  21329, 126898,  60315,    516,
           364, 126898,  28002,  92817,  60315,    516,    364,  34992,  44680,
           223,    112

In [19]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0])

레이블에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  54330,  31079,  85251,  90667,  21329,  80573,
        138520,  17877,  81718, 144059,  42039, 143604,  91145,    624,  12802,
        131866,  36055, 132760,  33704,  86038,  21329,  70943,    220,     16,
         59761,  18411,  86038, 129264, 130705,  10764,    252,    234,  28626,
         19969,  55673,  31079, 128732,  28733, 128472,     13, 130887, 135968,
        126550,  23573, 143603,  17877,    220,     16,  59761,  86038,  91145,
            13, 151645,    198, 151644,    872,    198, 128732,  51588,     25,
        126844,  55673,  70943,  56475, 130887, 139963, 132064,  19391,  64521,
         45130,    111,  33704, 139740,  31328,  19969,  35711,   5267,  14559,
         75132,  21329,     25,   2509, 144025,  53189,  79004,  21329,  83036,
         52959,    516,    364, 126746,  12802,  21329, 126898,  60315,    516,
           364, 126898,  28002,  92817,  60315,    516,    364,  34992,  44680,
           223,    11

In [20]:
# 토큰 디코딩 예시 (입력 텍스트가 어떻게 변환되었는지 확인)
decoded_text = processor.tokenizer.decode(batch["input_ids"][0])
print("\n디코딩된 텍스트:")
print(decoded_text)


디코딩된 텍스트:
<|im_start|>system
주어진 이미지와 질문을 바탕으로 답변하세요.
이때 정답은 선택지 중 1개를 선택해야하며 힌트가 주어질 수 있습니다. 가장 적절한 답을 1개 선택하세요.<|im_end|>
<|im_start|>user
질문: 다음 주 중에서 가장 북쪽에 있는 곳은 어디인가요?
선택지: ['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']
힌트: <|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_

In [21]:
from trl import SFTTrainer

trainer = SFTTrainer(
    # 앞서 로드한 Qwen2-VL 모델
    model=model,                   
    # SFTConfig를 통해 정의한 학습 설정
    args=args,                      
    # 학습에 사용할 데이터셋
    train_dataset=train_dataset,    
    # 데이터 배치 처리를 위한 collator 함수
    data_collator=collate_fn,       
    # 텍스트 필드 지정 (커스텀 collator 사용으로 빈 값)
    dataset_text_field="",          
    # LoRA 파인튜닝 설정
    peft_config=peft_config,        
    # 텍스트 토크나이징을 위한 토크나이저
    tokenizer=processor.tokenizer,  
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overf

In [ ]:
# 학습 시작, 모델은 출력 디렉토리에 저장됨
trainer.train()

# 모델 저장
trainer.save_model(args.output_dir)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
5,1.469300
10,0.833200
15,0.812700
20,0.642700
25,0.690400
30,0.552500
35,0.513900
40,0.447000
45,0.471400
50,0.433500


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


In [ ]:
from peft import PeftModel
from transformers import AutoProcessor, AutoModelForVision2Seq

adapter_path = "./output_dir/checkpoint-522"  # 학습된 어댑터 경로
base_model_id = "Qwen/Qwen2-VL-7B-Instruct"  # 기본 모델 ID
merged_path = "./merged"  # 병합된 모델을 저장할 경로

# 기본 모델 로드
model = AutoModelForVision2Seq.from_pretrained(base_model_id, low_cpu_mem_usage=True)

# LoRA와 기본 모델을 병합하고 저장
peft_model = PeftModel.from_pretrained(model, adapter_path)  # PEFT 모델 로드
merged_model = peft_model.merge_and_unload()  # 모델 병합
merged_model.save_pretrained(merged_path, safe_serialization=True, max_shard_size="2GB")  # 병합된 모델 저장

In [6]:
# 허깅페이스 로그인 
!huggingface-cli login --token 여러분들의KEY

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `test`


In [10]:
# 아래 커맨드로 모델 업로드
# !huggingface-cli upload-large-folder --repo-type=model Qwen2-VL-7B-instruct-KoScienceQA ./merged